In [ ]:
import shutil
import os
import numpy as np
import pandas as pd
import json
from m3inference import M3Inference
from m3inference import M3Twitter
import pprint
from langdetect import detect
import time
m3 = M3Inference()

10/14/2023 00:26:28 - INFO - m3inference.m3inference -   Version 1.1.5
10/14/2023 00:26:28 - INFO - m3inference.m3inference -   Running on cuda.
10/14/2023 00:26:28 - INFO - m3inference.m3inference -   Will use full M3 model.
10/14/2023 00:26:29 - INFO - m3inference.m3inference -   Model full_model exists at /g100/home/userexternal/mhabibi0/m3/models/full_model.mdl.
10/14/2023 00:26:29 - INFO - m3inference.utils -   Checking MD5 for model full_model at /g100/home/userexternal/mhabibi0/m3/models/full_model.mdl
10/14/2023 00:26:30 - INFO - m3inference.utils -   MD5s match.
10/14/2023 00:26:37 - INFO - m3inference.m3inference -   Loaded pretrained weight at /g100/home/userexternal/mhabibi0/m3/models/full_model.mdl


In [6]:
home_dir = '/g100/home/userexternal/mhabibi0/'
work_dir = '/g100_work/IscrC_mental'

hdata_dir = os.path.join(home_dir, 'Data')
wdata_dir = os.path.join(work_dir, 'data')
uc_dir = os.path.join(wdata_dir, 'user_classification')


wimg_dir = os.path.join(uc_dir, 'images')
wimg_test_dir = os.path.join(wimg_dir, 'test')
wimg_train_dir = os.path.join(wimg_dir, 'train')
wimg_de_dir = os.path.join(wimg_dir, 'de')

himg_dir = os.path.join(hdata_dir, 'images')
himg_test_dir = os.path.join(himg_dir, 'test')
himg_train_dir = os.path.join(himg_dir, 'train')
himg_de_dir = os.path.join(himg_dir, 'de')

In [8]:
os.listdir(himg_dir)

['preprocess.py', 'de', 'train', 'test']

In [3]:
# # copy test images to home

# himage_dir =os.path.join(hdata_dir, 'images', 'test')
# shutil.copytree(image_dir, himage_dir)


In [11]:
# copy train images to home
#shutil.copytree(wimg_test_dir, himg_test_dir)
shutil.copytree(wimg_train_dir, himg_train_dir)
shutil.copytree(wimg_de_dir, himg_de_dir)

'/g100/home/userexternal/mhabibi0/Data/images/de'

In [12]:
# resize images 
%cd /g100/home/userexternal/mhabibi0/Data/images/

/g100/home/userexternal/mhabibi0/Data/images


In [5]:
#!python preprocess.py --source_dir /g100/home/userexternal/mhabibi0/Data/images/test/ --output_dir /g100/home/userexternal/mhabibi0/Data/images/test/images_resized_v2/  

In [14]:
# !python preprocess.py --source_dir /g100/home/userexternal/mhabibi0/Data/images/train/ --output_dir /g100/home/userexternal/mhabibi0/Data/images/train/images_resized/  

In [16]:
# !python preprocess.py --source_dir /g100/home/userexternal/mhabibi0/Data/images/de/ --output_dir /g100/home/userexternal/mhabibi0/Data/images/de/images_resized/  

In [19]:
# bio lang
def detect_lang(bio_text):
    # languages that M3 supports
    list_langs = ['en', 'cs', 'fr', 'nl', 'ar', 'ro', 'bs', 'da', 'it', 'pt', 'no',
                  'es', 'hr', 'tr', 'de', 'fi', 'el', 'ru', 'bg', 'hu', 'sk', 'et', 
                  'pl', 'lv', 'sl', 'lt', 'ga', 'eu', 'mt', 'cy', 'rm', 'is', 'un']

    try:
        lang = detect(bio_text)
    except Exception as e:
        lang = 'it'  # default to 'it' in case of exceptions

    # if not supported by M3
    if lang not in list_langs:
        lang = 'it'
    
    return lang

## Test Data

In [18]:
# user age data
path  = os.path.join(uc_dir, 'data_for_models_test.pkl')
df_test = pd.read_pickle(path)

In [20]:
# transform the data to match M3 standard input
img_dir = os.path.join(himg_test_dir, 'images_resized')
# path to image
df_m3 = df_test[['user_id', 'masked_bio']]
df_m3.columns = ['id', 'description']
df_m3['id'] = df_m3['id'].astype(str)
df_m3['img_path'] = df_m3['id'].apply(lambda x: os.path.join(img_dir, str(x)+'.jpeg'))

# check if processed image exists
id_image_paths = df_m3['img_path'].values.tolist()
nonexist_paths_resized = [path for path in id_image_paths if not os.path.exists(path)]
nonexisting_ids = [path.split('/')[-1].split('.jpeg')[0] for path in nonexist_paths_resized]

# filter to ids having image
df_m3 = df_m3[~ df_m3['id'].isin(nonexisting_ids)]

# bio language
df_m3['lang'] = df_m3['description'].map(detect_lang)

df_m3['name'] = ''
df_m3['screen_name'] = ''

data_json = df_m3.to_dict('records')

/scratch_local/slurm_job.10984468/ipykernel_40907/2849860590.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['id'] = df_m3['id'].astype(str)
/scratch_local/slurm_job.10984468/ipykernel_40907/2849860590.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['img_path'] = df_m3['id'].apply(lambda x: os.path.join(img_dir, str(x)+'.jpeg'))


In [21]:
len(data_json)


1107

In [11]:
# from PIL import Image
# import matplotlib.pyplot as plt

# # Load the image
# uid = nonexist_ids[7]+'.jpg'
# image_path = os.path.join(himage_dir, uid)
# img = Image.open(image_path)

# # Display the image
# plt.imshow(img)
# plt.axis('off')  # To turn off axis numbers and ticks
# plt.show()

In [22]:
# find the results
from collections import OrderedDict

batch_size = 32  

results = OrderedDict()

# Process data_json in batches
for i in range(0, len(data_json), batch_size):
    batch = data_json[i:i+batch_size]
    result_batch = m3.infer(batch)
    results.update(result_batch)


10/14/2023 00:53:26 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:03<00:00,  2.00s/it]
10/14/2023 00:53:30 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.12it/s]
10/14/2023 00:53:31 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.45it/s]
10/14/2023 00:53:31 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.31it/s]
10/14/2023 00:53:32 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.72it/s]
10/14/2023 00:53:32 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.53it/s]
10/14/2023 00:53:33 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.40it/s]
10/14/2023 00:53:34 - INFO - m3inf

In [23]:
# save the results

path = os.path.join(uc_dir, 'M3_results_test_bio_image.json')
with open(path, 'w') as f:
    json.dump(results, f)

In [24]:
len(results.keys())

1107

#### Bio only 

In [25]:
# transform the data to match M3 standard input

df_m3 = df_test[['user_id', 'masked_bio']]
df_m3.columns = ['id', 'description']
df_m3['id'] = df_m3['id'].astype(str)

# remove cases with no bios
df_m3 = df_m3[ df_m3['description']!='']
# bio language
df_m3['lang'] = df_m3['description'].map(detect_lang)

df_m3['name'] = ''
df_m3['screen_name'] = ''

data_json = df_m3.to_dict('records')

/scratch_local/slurm_job.10984468/ipykernel_40907/762466409.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['id'] = df_m3['id'].astype(str)


In [26]:
# Process data_json in batches
results_bio_only = OrderedDict()
m3 = M3Inference(use_full_model=False)
for i in range(0, len(data_json), batch_size):
    batch = data_json[i:i+batch_size]
    result_batch = m3.infer(batch)
    results_bio_only.update(result_batch)

10/14/2023 00:54:51 - INFO - m3inference.m3inference -   Version 1.1.5
10/14/2023 00:54:51 - INFO - m3inference.m3inference -   Running on cuda.
10/14/2023 00:54:51 - INFO - m3inference.m3inference -   Will use text model. Note that as M3 was optimized to work well with both image and text data,                                     it is not recommended to use text only model unless you do not have the profile image.
10/14/2023 00:54:51 - INFO - m3inference.m3inference -   Model text_model exists at /g100/home/userexternal/mhabibi0/m3/models/text_model.mdl.
10/14/2023 00:54:51 - INFO - m3inference.utils -   Checking MD5 for model text_model at /g100/home/userexternal/mhabibi0/m3/models/text_model.mdl
10/14/2023 00:54:51 - INFO - m3inference.utils -   MD5s match.
10/14/2023 00:54:51 - INFO - m3inference.m3inference -   Loaded pretrained weight at /g100/home/userexternal/mhabibi0/m3/models/text_model.mdl
10/14/2023 00:54:51 - INFO - m3inference.dataset -   32 data entries loaded.
Predicti

In [28]:
list_keys = list(results_bio_only.keys())
len(list_keys)

519

In [29]:
# save the results
path = os.path.join(uc_dir, 'M3_results_test_bio_only.json')
with open(path, 'w') as f:
    json.dump(results_bio_only, f)

## Train Data

In [30]:
# user age data
path  = os.path.join(uc_dir, 'data_for_models_train.pkl')
df_train = pd.read_pickle(path)

In [31]:
# transform the data to match M3 standard input
img_dir = os.path.join(himg_train_dir, 'images_resized')
# path to image
df_m3 = df_train[['user_id', 'masked_bio']]
df_m3.columns = ['id', 'description']
df_m3['id'] = df_m3['id'].astype(str)
df_m3['img_path'] = df_m3['id'].apply(lambda x: os.path.join(img_dir, str(x)+'.jpeg'))

# check if processed image exists
id_image_paths = df_m3['img_path'].values.tolist()
nonexist_paths_resized = [path for path in id_image_paths if not os.path.exists(path)]
nonexisting_ids = [path.split('/')[-1].split('.jpeg')[0] for path in nonexist_paths_resized]

# filter to ids having image
df_m3 = df_m3[~ df_m3['id'].isin(nonexisting_ids)]

# bio language
df_m3['lang'] = df_m3['description'].map(detect_lang)

df_m3['name'] = ''
df_m3['screen_name'] = ''

data_json = df_m3.to_dict('records')

/scratch_local/slurm_job.10984468/ipykernel_40907/2727949520.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['id'] = df_m3['id'].astype(str)
/scratch_local/slurm_job.10984468/ipykernel_40907/2727949520.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['img_path'] = df_m3['id'].apply(lambda x: os.path.join(img_dir, str(x)+'.jpeg'))


In [32]:
len(data_json)


15337

In [41]:
# find the results
from collections import OrderedDict

batch_size = 32  

results = OrderedDict()

# Process data_json in batches
for i in range(0, len(data_json), batch_size):
    batch = data_json[i:i+batch_size]
    result_batch = m3.infer(batch)
    results.update(result_batch)


In [34]:
len(results.keys())

15337

In [35]:
# save the results

path = os.path.join(uc_dir, 'M3_results_train_bio_image.json')
with open(path, 'w') as f:
    json.dump(results, f)

## German Data

In [38]:
path = os.path.join(uc_dir, 'german_data')
os.listdir(path)

['data_for_models_german_data.pkl',
 'german_tweets.pkl',
 'german_users_with_birthyear.pkl']

In [39]:
# user age data
path  = os.path.join(uc_dir, 'german_data', 'data_for_models_german_data.pkl')
df_de = pd.read_pickle(path)

In [42]:
# transform the data to match M3 standard input
img_dir = os.path.join(himg_de_dir, 'images_resized')
# path to image
df_m3 = df_de[['user_id', 'masked_bio']]
df_m3.columns = ['id', 'description']
df_m3['id'] = df_m3['id'].astype(str)
df_m3['img_path'] = df_m3['id'].apply(lambda x: os.path.join(img_dir, str(x)+'.jpeg'))

# check if processed image exists
id_image_paths = df_m3['img_path'].values.tolist()
nonexist_paths_resized = [path for path in id_image_paths if not os.path.exists(path)]
nonexisting_ids = [path.split('/')[-1].split('.jpeg')[0] for path in nonexist_paths_resized]

# filter to ids having image
df_m3 = df_m3[~ df_m3['id'].isin(nonexisting_ids)]

# bio language
df_m3['lang'] = df_m3['description'].map(detect_lang)

df_m3['name'] = ''
df_m3['screen_name'] = ''

data_json = df_m3.to_dict('records')

/scratch_local/slurm_job.10984468/ipykernel_40907/290704806.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['id'] = df_m3['id'].astype(str)
/scratch_local/slurm_job.10984468/ipykernel_40907/290704806.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['img_path'] = df_m3['id'].apply(lambda x: os.path.join(img_dir, str(x)+'.jpeg'))


In [43]:
len(data_json)


420

In [44]:
# find the results
from collections import OrderedDict

batch_size = 32  

results = OrderedDict()

# Process data_json in batches
for i in range(0, len(data_json), batch_size):
    batch = data_json[i:i+batch_size]
    result_batch = m3.infer(batch)
    results.update(result_batch)


10/14/2023 01:13:38 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.52it/s]
10/14/2023 01:13:39 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.54it/s]
10/14/2023 01:13:39 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.53it/s]
10/14/2023 01:13:40 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.56it/s]
10/14/2023 01:13:40 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.55it/s]
10/14/2023 01:13:41 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.56it/s]
10/14/2023 01:13:41 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.52it/s]
10/14/2023 01:13:42 - INFO - m3inf

In [45]:
len(results.keys())

420

In [46]:
# save the results

path = os.path.join(uc_dir, 'M3_results_de_bio_image.json')
with open(path, 'w') as f:
    json.dump(results, f)